# Графики

In [ ]:
from tqdm import tqdm
quant_hist = 2

splitter = TimeRangeSplitter(
    test_size="7D",
    n_splits=quant_hist,
    filter_already_seen=True,
)


splits = splitter.get_test_fold_borders(dataset.interactions)

print(splits)

first_date = splits[0][0]

# Настраиваем стиль (опционально)
sns.set_style("whitegrid")
sns.set_palette("bright")

# Создаем фигуру и оси
fig, ax = plt.subplots(figsize=(10, 6))

recoms_list = []
for i in tqdm(range(quant_hist)):
    second_date = splits[i][1]
    print(second_date)
    als_dataset =  Dataset.construct(
    interactions_df=interactions_first_period[(pd.to_datetime(interactions_first_period['datetime']) <= pd.to_datetime(first_date)) & (pd.to_datetime(interactions_first_period['datetime']) <= pd.to_datetime(second_date))],
    )


    als_model = als.fit(dataset)
    recoms = als_model.recommend(users=dataset.user_id_map.external_ids[:30], dataset=dataset, filter_viewed=True, k=100)
    recoms_list.append(recoms["score"])
    # Строим первую гистограмму с прозрачностью
    sns.histplot(recoms["score"], ax=ax, label=f'Распределение скоров {i}',  kde=True, bins=50) # Добавлен kde=True для сглаживания



# Добавляем заголовок, подписи осей и легенду
ax.set_title('Несколько гистограмм на одном графике')
ax.set_xlabel('Значения')
ax.set_ylabel('Частота')
ax.legend()

# Отображаем график
plt.show()


# Calc NDCG

In [ ]:
from sklearn.metrics import ndcg_score
import numpy as np

true_relevance = pd.read_csv("data/true_relevance.csv")

val_scores = X_val[["user_id", "item_id"]]
val_scores['prediction'] = model.predict(val_pool)

all_scores = val_scores.merge(true_relevance, on=["user_id", "item_id"])
grouped = all_scores.groupby("user_id").aggregate(lambda x: list(x))


pred = []
true_pred = []
for a in grouped.itertuples():
    pred.append(a.prediction)
    true_pred.append(a.true_relevance)

ndcg_score(true_pred, pred)

# True scores

# SHAP

In [ ]:
import shap

# Создайте экземпляр SHAP
explainer = shap.TreeExplainer(model)

# Получите SHAP-значения для данных
shap_values = explainer(X_val.drop(columns=["user_id", "item_id"]))

# Постройте SHAP-график
shap.plots.beeswarm(shap_values)


# Правильный датасет

In [ ]:
target_df = interactions_second_period[["user_id", "item_id"]].drop_duplicates()
target_df["target"] = interactions_second_period["action"].map({"click": 1, "show": 0})


all_features = pd.merge(users["user_id"], items["item_id"], how="cross")

# Рассчет фичей

In [ ]:
def calc_counters(interactions, user_df, item_df):
    """
    Calculates features for each user-item pair in the dataframe.
    """
    
    # target_df = interactions[["user_id", "item_id"]]
    # target_df["target"] = interactions["action"].map({"click": 1, "show": 0})
    
    # print(target_df)
    
    # user_counters = interactions["user_id"].value_counts().reset_index().rename(columns={"count": "user_count"})
    # item_counters = target_df.drop(columns="user_id").groupby("item_id").mean().rename(columns={"target": "ctr"})

    # target_df.drop(columns="user_id").groupby("item_id").mean().rename(columns={"target": "ctr"})
    
    # features_users = user_df.merge(user_counters, on="user_id")
    # features_items = item_df.merge(item_counters, on="item_id")
    
    # return features_users, features_items
    
    return user_df, item_df
